In [216]:
import IPython.core.display as di

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

Toggle code

# Project: Machine Learning for Amazon Repricing 

Date: May 10, 2017

Authors: Thanh Binh Le, Georgiana Ifrim


In this report we analyse a dataset from Xsellco Ltd delivered to us in January 2017.
The dataset consists of 119,709 XML files, each XML file is associated with one competition 
on the Amazon Marketplace platform.
A competition is a set of sellers selling the same product on the Amazon platform, at a given point in time.
Each seller within a competition places an offer for selling the product at a given price, and with different constraints, such as shipping related constraints (e.g., availability, min time to ship), product related (e.g., new or used), seller profile (positive rating, number of ratings, etc.). Each competition should in principle only have one winner (winning offer), but we actually find that there are several competitions without any winners or with several winners. 

We discuss several problems we found with the XML data format (e.g., different XML structure) and the competitions (e.g., no winners or multiple winners per competition), and analyse the data characteristics for each of the 7 markets covered by the dataset provided.
We discuss how we extract the data from the XML files into a format more amenable for data analytics techniques,
as well as the steps for preparing the data for further analysis and for prediction modeling.

We find that each market has different data characteristics, with different problems being dominant and different feature importance for predicting the winner. This has implications for how we model the data in each market and for how we may advise customers to update their offer to improve their chances of winning the competition.

## <font color='darkblue'>From XML to CSV dataset</font>

One first step is to parse the XML files into a single CSV dataset that can be use for further data analysis.
We do this by parsing every single XML file, and turning each tag <Offer> in the XML file, into a row of the CSV file. This row than describes the features of an offer and the outcome of the offer in the target feature IsBuyBoxWinner.

###  <font color='darkblue'>Problem1: Some XML files do not have the correct structure</font>

Out of 119,709 XML files,  we can correctly parse 112,710 XML files.
We have investigated why the parsing fails on some XML files and found that for those files the XML structure is different.

An example correct and incorrect XML structure is shown below.

In [3]:

"""
================================
Import libraries & functions
================================
"""
#print(__doc__)
# Import pandas, numpy and matplotlib libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from xml.etree import ElementTree 
from lxml import etree
import warnings
from pylab import rcParams
#from skimage import io
import seaborn as sb
import timeit as time
from tabulate import tabulate
from IPython.display import clear_output
%matplotlib inline
warnings.filterwarnings('ignore')


In [151]:
# def pieplotting(x,y,explode,title,message):
#     colors = ['yellowgreen','red','gold','lightskyblue','white']
#     porcent = 100.*y/y.sum()
#     patches = plt.pie(y, colors=colors, startangle=90,counterclock=False, radius=1.2,shadow=True,explode=explode)
#     labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(x, porcent)]
#     legend = plt.legend(patches[0], labels, loc='left center', bbox_to_anchor=(-0.1, 1.),fontsize=15)
#     legend.get_frame().set_facecolor('none')
#     plt.axis('equal')
#     plt.suptitle(title)
#     print(message)
#     plt.show()
    
import math

def list_columns(obj, cols=4, columnwise=True, gap=4):
    """
    Print the given list in evenly-spaced columns.

    Parameters
    ----------
    obj : list
        The list to be printed.
    cols : int
        The number of columns in which the list should be printed.
    columnwise : bool, default=True
        If True, the items in the list will be printed column-wise.
        If False the items in the list will be printed row-wise.
    gap : int
        The number of spaces that should separate the longest column
        item/s from the next column. This is the effective spacing
        between columns based on the maximum len() of the list items.
    """

    sobj = [str(item) for item in obj]
    if cols > len(sobj): cols = len(sobj)
    max_len = max([len(item) for item in sobj])
    if columnwise: cols = int(math.ceil(float(len(sobj)) / float(cols)))
    plist = [sobj[i: i+cols] for i in range(0, len(sobj), cols)]
    if columnwise:
        if not len(plist[-1]) == cols:
            plist[-1].extend(['']*(len(sobj) - len(plist[-1])))
        plist = zip(*plist)
    printer = '\n'.join([
        ''.join([c.ljust(max_len + gap) for c in p])
        for p in plist])
    print(printer)

In [5]:
# Load the normal XML
x = etree.parse("1454732838.xml")
print("XML file: 1454732838.xml\n\n")
print(etree.tostring(x, pretty_print = True,encoding="unicode")[:505])
#print(etree.tostring(x, pretty_print = True,encoding="unicode"))

XML file: 1454732838.xml


<Notification>
<NotificationMetaData>
	<NotificationType>AnyOfferChanged</NotificationType>
	<PayloadVersion>1.0</PayloadVersion>
	<UniqueId>864d3c39-83f0-4e36-b5d9-ee7b6cd23a8e</UniqueId>
	<PublishTime>2016-02-06T04:14:57.588Z</PublishTime>
	<SellerId>A3NHQRVGEOAUAF</SellerId>
	<MarketplaceId>A2EUQ1WTGCTBG2</MarketplaceId>
</NotificationMetaData>
<NotificationPayload>
	<AnyOfferChangedNotification>
		<OfferChangeTrigger>
			<MarketplaceId>A2EUQ1WTGCTBG2</MarketplaceId>
			<ASIN>B00CG5EH4M</ASIN>
			


In [6]:
#Load the problem XML 
x = etree.parse("1454627640.xml")
print("XML file: 1454627640.xml\n\n")
print(etree.tostring(x, pretty_print = True,encoding="unicode")[:522])

XML file: 1454627640.xml


<GetLowestOfferListingsForSKUResponse xmlns="http://mws.amazonservices.com/schema/Products/2011-10-01">
  <GetLowestOfferListingsForSKUResult status="Success">
    <AllOfferListingsConsidered>false</AllOfferListingsConsidered>
    <Product xmlns="http://mws.amazonservices.com/schema/Products/2011-10-01" xmlns:ns2="http://mws.amazonservices.com/schema/Products/2011-10-01/default.xsd">
      <Identifiers>
        <MarketplaceASIN>
          <MarketplaceId>A2EUQ1WTGCTBG2</MarketplaceId>
          <ASIN>B004M0LFSA</ASIN>


**Problem1:** Some XML files have a different format. The difference appears in both tagnames and the structure of XML. 

**Solution:** Ignore XML files with different format.

       Summary:    
       ---------------------------------------------------------------------------
       Total files: 119,709 files    
       Parsed files: 112,710  files      
       Missing:      6,999 files         
       ---------------------------------------------------------------------------

### <font color='darkblue'>From XML tags to CSV columns.</font>
All the offers under the tag **Offers** become rows in the CSV file. 
Since each XML file has up to 20 offers, this means that for each competition we generate 
up to 20 rows in the CSV file.
Each **Offer** tag becomes a row in the CSV file, where each tag or subtag, becomes a column name.
We give details below for the number of rows and columns generated by parsing 112,710 XML files.
For each offer there are at most 17 distinct tags and subtags which generate 17 distinct columns. 
Beside that, we also add to CSV rows the additional info in each XML file, such as the time of offers change, the market place Id, the product Id, and the seller Id.
We modify the values of some tags when converting to CSV as follows:

**TimeOfferChange** is split into: Year,	Month,	Day,	Hour,	Minute,	Second,	Milisecond

**ConditionNotes** is missing for many offers, so we turn it into one feature with value 1 (available) or -1 (missing). We currently do not exploit the text of ConditionNotes.

**IsBuyBoxWinner** tag becomes a CSV column with the same name, but takes values 1 (positive example, winner is True) or -1 (negative example, winner is False).

All other CSV columns are kept with raw values as present in the XML file as **CompetitionName**.
Parsing 112,710 XML files results in a CSV file with about 1.3M rows (examples) with 28 columns (features).


## <font color='darkblue'>Analyse CSV dataset</font>

In this section we analyse the raw data and apply a set of transformations:

1. We drop duplicated rows or columns. (**Q: Why do duplicated rows exist?**)
2. We drop columns with missing data.
3. We check the cardinality of each column and drop constant columns.

We then check descriptive statistics for each feature (e.g., min, max, deviation, etc.)

In [7]:
"""
================================
    Reading dataset
================================
"""
#print(__doc__)
df_ful = pd.read_csv(u"./100k/full_data.csv", error_bad_lines=False)

### <font color='darkblue'>Number of rows, columns</font>


In [8]:
# """
# ================================
#     Number of rows, columns
# ================================
# """
# print(__doc__)
print("Size of data (rows, columns): ({0},{1})".format(df_ful.shape[0],df_ful.shape[1]))

Size of data (rows, columns): (1304661,28)


In [9]:
# """
# ================================
#     Show columns
# ================================
# """
# print(__doc__)
print('Column features: \n================================')
list_columns(df_ful.columns.values)

Column features: 
IsBuyBoxWinner            Minute                    ListingCurrency           ShippingTime_maxHours     
MarketplaceId             Second                    SellerFeedbackRating      ShippingTime_availtype    
ProductId                 Milisecond                SellerFeedbackCount       ShipsDomestically         
Year                      ConditionNotes            SellerId                  ShipsFromCountry          
Month                     IsFeaturedMerchant        ShippingPrice              ShipsFromState           
Day                       IsFulfilledByAmazon       ShippingCurrency          SubCondition              
Hour                      ListingPrice              ShippingTime_minHours     CompetitionName           


In [75]:
"""
================================
    First 10 rows, all columns
================================
"""
print(__doc__)
from IPython.display import display
pd.set_option('max_colwidth',15)
display(df_ful.head(10))


    First 10 rows, all columns



,IsBuyBoxWinner,MarketplaceId,ProductId,Year,Month,Day,Hour,Minute,Second,Milisecond,...,ShippingPrice,ShippingCurrency,ShippingTime_minHours,ShippingTime_maxHours,ShippingTime_availtype,ShipsDomestically,ShipsFromCountry,ShipsFromState,SubCondition,CompetitionName
0,1,APJ6JRA9NG5V4,B01JGPBFOO,2017,2,2,18,31,46,402,...,0.0,EUR,24,48,NOW,1,IT,NaN,new,0095b1ed-b1...
1,-1,APJ6JRA9NG5V4,B01JGPBFOO,2017,2,2,18,31,46,402,...,0.0,EUR,24,48,NOW,1,IT,NaN,new,0095b1ed-b1...
2,1,APJ6JRA9NG5V4,B01JGPBFOO,2017,2,2,18,13,50,977,...,0.0,EUR,24,48,NOW,1,IT,NaN,new,00b27eb6-4f...
3,-1,APJ6JRA9NG5V4,B01JGPBFOO,2017,2,2,18,13,50,977,...,0.0,EUR,24,48,NOW,1,IT,NaN,new,00b27eb6-4f...
4,1,APJ6JRA9NG5V4,B01NAWL24O,2017,2,2,23,29,34,912,...,0.0,EUR,24,48,NOW,1,IT,NaN,new,03d942d6-41...
5,-1,APJ6JRA9NG5V4,B01NAWL24O,2017,2,2,23,29,34,912,...,0.0,EUR,24,48,NOW,1,IT,NaN,new,03d942d6-41...
6,1,APJ6JRA9NG5V4,B01JGPGIJQ,2017,2,2,23,2,8,243,...,0.0,EUR,24,48,NOW,1,IT,NaN,new,051ba696-d6...
7,-1,APJ6JRA9NG5V4,B01JGPGIJQ,2017,2,2,23,2,8,243,...,0.0,EUR,24,48,NOW,1,IT,NaN,new,051ba696-d6...
8,1,APJ6JRA9NG5V4,B01NBY4YQG,2017,2,2,23,6,4,396,...,0.0,EUR,24,48,NOW,1,IT,NaN,new,05e7f80e-02...
9,-1,APJ6JRA9NG5V4,B01NBY4YQG,2017,2,2,23,6,4,396,...,0.0,EUR,24,48,NOW,1,IT,NaN,new,05e7f80e-02...


### <font color='darkblue'>Duplicated in rows, columns</font>

In [77]:
"""
================================
   Duplicated rows, columns
================================
"""
print(__doc__)

sizeold = df_ful.shape[0]
# drop duplicated rows
df_ful.drop_duplicates(inplace=True) # check and clean the duplication
sizenew = df_ful.shape[0]
no_duplicated = sizeold - sizenew
df_ful.to_csv('./100k/full-clean.csv',sep = ',',index = False)
print('Size of Raw file: ' + str('{:,}'.format(sizeold)))
print('Size of Clean file: ' + str('{:,}'.format(sizenew)))
print('Duplication: ' + str('{:,}'.format(no_duplicated)) + ' samples')

# drop duplicated column
df_ful = df_ful.drop("ListingCurrency", 1) 


   Duplicated rows, columns

Size of Raw file: 1,304,661
Size of Clean file: 1,303,556
Duplication: 1,105 samples


### <font color='darkblue'>Missing values per column</font>

In [78]:
"""
================================
   Missing values per column
================================
"""

print(__doc__)
#Create a new function: 
def num_missing(x):
    return x.name, sum(x.isnull())

#Applying per column:
print("Total rows: " + str('{:,}'.format(sizenew)))
print("------------------------------------------------")
list_columns(df_ful.apply(num_missing),cols=3)


   Missing values per column

Total rows: 1,303,556
------------------------------------------------
('IsBuyBoxWinner', 0)            ('Milisecond', 0)                ('ShippingCurrency', 0)          
('MarketplaceId', 0)             ('ConditionNotes', 0)            ('ShippingTime_minHours', 0)     
('ProductId', 0)                 ('IsFeaturedMerchant', 0)        ('ShippingTime_maxHours', 0)     
('Year', 0)                      ('IsFulfilledByAmazon', 0)       ('ShippingTime_availtype', 0)    
('Month', 0)                     ('ListingPrice', 0)              ('ShipsDomestically', 0)         
('Day', 0)                       ('SellerFeedbackRating', 0)      ('ShipsFromCountry', 275930)     
('Hour', 0)                      ('SellerFeedbackCount', 0)       (' ShipsFromState', 1303018)     
('Minute', 0)                    ('SellerId', 0)                  ('SubCondition', 0)              
('Second', 0)                    ('ShippingPrice', 0)             ('CompetitionName', 0)          

In [79]:
"""
================================
Drop the missing values columns
================================
"""
print(__doc__) 
df_ful = df_ful.drop("ShipsFromCountry", 1) 
df_ful = df_ful.drop(" ShipsFromState", 1) 
list_columns(list(df_ful),cols=4)


Drop the missing values columns

IsBuyBoxWinner            Minute                    SellerFeedbackRating      ShippingTime_availtype    
MarketplaceId             Second                    SellerFeedbackCount       ShipsDomestically         
ProductId                 Milisecond                SellerId                  SubCondition              
Year                      ConditionNotes            ShippingPrice             CompetitionName           
Month                     IsFeaturedMerchant        ShippingCurrency                                    
Day                       IsFulfilledByAmazon       ShippingTime_minHours                               
Hour                      ListingPrice              ShippingTime_maxHours                               


### <font color='darkblue'>Unique values for each feature</font>

In [82]:
"""
================================
Features and unique values
================================
"""
print(__doc__)
# Check the unique values inside of features
#Look at the numerical features only
numeric_columns_full = df_ful.select_dtypes(['int64', 'float64']).columns
print('List of numeric features:')
list_columns(numeric_columns_full.values)
#Look at the categorical features only
print('---------------------')
category_columns_full = df_ful.select_dtypes(['category', 'object']).columns
print('List of category features:')
list_columns(category_columns_full.values)
print('\n \n---------------------------------------------')
print("%-*s: %s" % (22,'Feature','UniqueValues'))
print('---------------------------------------------')
for column in numeric_columns_full:
    print("%-*s: %s" % (22,str(column),str(len(df_ful[column].unique()))))
for column in category_columns_full:
    print("%-*s: %s" % (22,str(column),str(len(df_ful[column].unique()))))


Features and unique values

List of numeric features:
IsBuyBoxWinner           Minute                   IsFulfilledByAmazon      ShippingTime_minHours    
Year                     Second                   ListingPrice             ShippingTime_maxHours    
Month                    Milisecond               SellerFeedbackRating     ShipsDomestically        
Day                      ConditionNotes           SellerFeedbackCount                               
Hour                     IsFeaturedMerchant       ShippingPrice                                     
---------------------
List of category features:
MarketplaceId             SellerId                  ShippingTime_availtype    CompetitionName           
ProductId                 ShippingCurrency          SubCondition                                        

 
---------------------------------------------
Feature               : UniqueValues
---------------------------------------------
IsBuyBoxWinner        : 2
Year                  :

### <font color='darkblue'>Drop the constant columns</font>

In [83]:
"""
================================
  Drop the constant columns
================================
"""
print(__doc__) 
df_ful = df_ful.drop("ShipsDomestically", 1) 
list_columns(list(df_ful))


  Drop the constant columns

IsBuyBoxWinner            Hour                      IsFulfilledByAmazon       ShippingCurrency          
MarketplaceId             Minute                    ListingPrice              ShippingTime_minHours     
ProductId                 Second                    SellerFeedbackRating      ShippingTime_maxHours     
Year                      Milisecond                SellerFeedbackCount       ShippingTime_availtype    
Month                     ConditionNotes            SellerId                  SubCondition              
Day                       IsFeaturedMerchant        ShippingPrice             CompetitionName           


### <font color='darkblue'>Data quality report</font>

In [84]:
"""
================================
Features descriptive statistics
================================
"""
print(__doc__)
#Prepare a table with descriptive statistics for all the continuous features.
#Look at the numerical features only
numeric_columns_full = df_ful.select_dtypes(['int64', 'float64']).columns
print('List of numeric features:')
list_columns(numeric_columns_full.values)

print("\n\n---------------------  -----------  -----------  ------------  -------  ------  -------  -------  ------")
print(tabulate(df_ful[numeric_columns_full].describe().T,headers="keys",tablefmt="simple"))
print("---------------------  -----------  -----------  ------------  -------  ------  -------  -------  ------")


Features descriptive statistics

List of numeric features:
IsBuyBoxWinner           Minute                   IsFulfilledByAmazon      ShippingTime_minHours    
Year                     Second                   ListingPrice             ShippingTime_maxHours    
Month                    Milisecond               SellerFeedbackRating                              
Day                      ConditionNotes           SellerFeedbackCount                               
Hour                     IsFeaturedMerchant       ShippingPrice                                     


---------------------  -----------  -----------  ------------  -------  ------  -------  -------  ------
                             count         mean           std      min     25%      50%      75%     max
---------------------  -----------  -----------  ------------  -------  ------  -------  -------  ------
IsBuyBoxWinner         1.30356e+06    -0.82079       0.57123     -1       -1      -1       -1          1
Year         

#  <font color='darkblue'>Designing & Implementing Features</font>

## <font color='darkblue'>The data's problems</font>

The problems we discovered in the data provided (119K XML files) are described as follows:

        1) Several XML files not in the same format to the other ones 
        (XMLFormat; discussed above as Problem1)
        
        2) More than one winner in one competition time (MultiWin)

        3) No winner for some competitions (ZeroWin)

        4) Same seller different offers in one competition (MultiSeller)

        5) Small number of offers (less than 3) in some competitions (FewOffer)

Below we first split the full CSV dataset according to each market, then analyse the distribution of problems 2-5 for each such dataset. We want to study the data characteristics, for each market.


## <font color='darkblue'>Split data by Market ID</font>

<font color='darkred'>We analyse for each market: </font>
<div class="alert alert-block alert-warning">
<p>a. How many products are being sold? (to get a feel for market size)</p>
<p>b. How many competitions are there? (to get a fell for dynamics of market)</p>
<p>c. How many problems are present across products? (to check if problems come mostly from a few  products)</p> 
<p>d. How many problems are present across competitions? (to check types of problems and how many  competitions are affected)</p>
<p>e. We bin the products according to number of competitions (to check how many products have many offers, or few offers).</p>
<p>f. We check problem types for each type of bin (to see if intensive or little competition leads to particular problems).</p>
</div>

In [152]:
# path
import os
# Function to get list of filename in directory
def get_filepaths(directory):
    file_paths = []  # List which will store all of the full filepaths.

    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            # Join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)  # Add it to the list.

    return file_paths  # Self-explanatory.
data_path = u'./Market'
path = get_filepaths(data_path)
listname = []

index = ['France','United Kingdom','Germany','Spain','India','Italy','US']
dataframe = pd.DataFrame([],index=index, columns=['Note','#Product','#Competition',\
                                                  '#MultiWin\n/#Competition', \
                                                  '#ZeroWin\n/#Competition',\
                                                  '#MultiSeller\n/#Competition',\
                                                  '#FewOffer\n/#Competition',\
                                                  '#MultiWin\n/#Product', \
                                                  '#ZeroWin\n/#Product',\
                                                  '#MultiSeller\n/#Product',\
                                                  '#FewOffer\n/#Product'])
dataframe.index.name = 'Markets'
for f in path:
    if f.endswith(".csv"):
        name = (os.path.splitext(os.path.basename(f))[0])
        exec(name + "= pd.read_csv(f,error_bad_lines=False)")
        listname.append(name)
dataframe["Note"] = listname
# product
p1 = len(Market1_data["ProductId"].unique())
p2 = len(Market2_data["ProductId"].unique())
p3 = len(Market3_data["ProductId"].unique())
p4 = len(Market4_data["ProductId"].unique())
p5 = len(Market5_data["ProductId"].unique())
p6 = len(Market6_data["ProductId"].unique())
p7 = len(Market7_data["ProductId"].unique())
# competition
c1 = len(Market1_data["CompetitionName"].unique())
c2 = len(Market2_data["CompetitionName"].unique())
c3 = len(Market3_data["CompetitionName"].unique())
c4 = len(Market4_data["CompetitionName"].unique())
c5 = len(Market5_data["CompetitionName"].unique())
c6 = len(Market6_data["CompetitionName"].unique())
c7 = len(Market7_data["CompetitionName"].unique())
dataframe["#Product"] = [p1,p2,p3,p4,p5,p6,p7]
dataframe["#Competition"] = [c1,c2,c3,c4,c5,c6,c7]

In [153]:
import sys
def checkPrbl(df):
        ## Code checking 4 problems
    groups = df.groupby(['CompetitionName'])
    i =0
    l = len(groups)
    ZeroWin =0 
    MultiWin = 0
    MultiSeller=0
    FewOffer=0
    for competition_name, group in (groups):
        l1= len(group)
        numberOffers = group['IsBuyBoxWinner'].count()
        # b
        tmpSum = group.IsBuyBoxWinner.sum()
        if (abs(tmpSum) == int(numberOffers)):
            ZeroWin += 1
        # c
        tmpSum = group.IsBuyBoxWinner.sum()
        if (abs(tmpSum) < int(numberOffers) - 2):
            MultiWin += 1
        # d
        group = group.drop("ListingPrice", 1) 
        group = group.drop("ShippingPrice", 1) 
        SellerId = group['SellerId']
        group_duplicated = SellerId[group.duplicated() == True]
        group_duplicated = group_duplicated.unique()
        if len(group_duplicated) != 0:
            MultiSeller += 1
        # e
        if(int(numberOffers) <= 3): # threshold is = 3
            FewOffer += 1
        i+=1
#         sys.stdout.write("\r{0}:{1}:{2}".format(i,l,l1))
#         sys.stdout.flush()
    
    return ([l,MultiWin,ZeroWin,MultiSeller,FewOffer])   

In [155]:
## Market 1
L1 =[]
i=1
groups = Market1_data.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    L1.append(t)
    i+=1
L1 = np.reshape(L1,(l, 5))
SS1 = sum(L1)
S1 = sum(L1!=0)
clear_output()

In [156]:
## Market 2
L2 =[]
i=1
groups = Market2_data.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    L2.append(t)
    i+=1
L2 = np.reshape(L2,(l, 5))
SS2 = sum(L2)
S2 = sum(L2!=0)
clear_output()

In [157]:
## Market 3
L3 =[]
i=1
groups = Market3_data.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    L3.append(t)
    i+=1
L3 = np.reshape(L3,(l, 5))
SS3 = sum(L3)
S3 = sum(L3!=0)
clear_output()

In [158]:
## Market 4
L4 =[]
i=1
groups = Market4_data.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    L4.append(t)
    i+=1
L4 = np.reshape(L4,(l, 5))
SS4 = sum(L4)
S4 = sum(L4!=0)
clear_output()

In [159]:
## Market 5
L5 =[]
i=1
groups = Market5_data.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    L5.append(t)
    i+=1
L5 = np.reshape(L5,(l, 5))
SS5 = sum(L5)
S5 = sum(L5!=0)
clear_output()

In [160]:
## Market 6
L6 =[]
i=1
groups = Market6_data.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    L6.append(t)
    i+=1
L6 = np.reshape(L6,(l, 5))
SS6= sum(L6)
S6 = sum(L6!=0)
clear_output()

In [161]:
## Market 7
L7 =[]
i=1
groups = Market7_data.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    L7.append(t)
    i+=1
L7 = np.reshape(L7,(l, 5))
SS7 = sum(L7)
S7 = sum(L7!=0)
clear_output()

In [162]:
A1 = np.reshape([SS1,SS2,SS3,SS4,SS5,SS6,SS7],(7,5))

In [163]:
A2 = np.reshape([S1,S2,S3,S4,S5,S6,S7],(7,5))

## <font color='darkblue'>Products and problems per market</font>
We observe that France, Italy and US are larger markets than the other (wrt products sold).

Regarding problems identified across products:

**Multiwin:** Only France and Italy have this problem. France has 97 (of 460) products with at least one competition with more than one winner. Italy has 57/1349 products with this problem.

**Zerowin:**  All markets have products with at least one competition without a winner.

**Multiseller:** France and Italy have a more significant seller cheating problem. For Frace 35/460 products have at least one competition whith same seller placing different offers in the same competition. 

**FewOffers:** Almost all markets have at least 50% products with few offers, and most markets have close to 100% products with few offers.

In [164]:
## Add into dataframe
dataframe["#MultiWin\n/#Competition"] = A1[:,1]
dataframe["#ZeroWin\n/#Competition"] = A1[:,2]
dataframe["#MultiSeller\n/#Competition"] = A1[:,3]
dataframe["#FewOffer\n/#Competition"] = A1[:,4]

In [165]:
## Add into dataframe
dataframe["#MultiWin\n/#Product"] = A2[:,1]
dataframe["#ZeroWin\n/#Product"] = A2[:,2]
dataframe["#MultiSeller\n/#Product"] = A2[:,3]
dataframe["#FewOffer\n/#Product"] = A2[:,4]

In [166]:
"""
================================
Prbs per product for 7 markets
================================
"""
print(__doc__)
## split into dataframe by products
dataframe_prod = dataframe.ix[:,['Note','#Product','#MultiWin\n/#Product','#ZeroWin\n/#Product'\
                                 ,'#MultiSeller\n/#Product','#FewOffer\n/#Product']]
dataframe_prod.columns=['Note','#Product','#MultiWin','#ZeroWin','#MultiSeller','#FewOffer']
print(tabulate(dataframe_prod,headers='keys', tablefmt='simple'))


Prbs per product for 7 markets

Markets         Note            #Product    #MultiWin    #ZeroWin    #MultiSeller    #FewOffer
--------------  ------------  ----------  -----------  ----------  --------------  -----------
France          Market1_data         460           97         117              35          202
United Kingdom  Market2_data         117            0          91               0          117
Germany         Market3_data         130            0         111               1          130
Spain           Market4_data         163            0         129               2          162
India           Market5_data         134            0         111               4          134
Italy           Market6_data        1349           57         209              65          785
US              Market7_data         945            0         115               2          936


## <font color='darkblue'>Competitions and problems per market</font>

We observe that France is a very dynamic market, followed by Italy and the US.

Regarding problems identified across competitions:

**Multiwin:** France and Italy have this problem. For France almost 10% of all competitions have more than one winner.

**Zerowin:**  All markets have competitions without a winner. This problem is significant, it occurs in more than 50% of competitions for some markets (e.g., UK). This implies that simply throwing out these competitions will lead to significant data loss.

**Multiseller:** France and Italy have a fairly significant seller cheating problem. For France, 2% (1706/79470) of all competitions have the same seller placing different offers in the same competition. 

**FewOffers:** Almost all markets have at least 50% products with few offers, and most markets have close to 100% products with few offers.

In [167]:
"""
==================================
Prbs per competition for 7 markets
==================================
"""
print(__doc__)
## split into dataframe by competitions
dataframe_comp = dataframe.ix[:,['Note','#Competition','#MultiWin\n/#Competition',\
                                 '#ZeroWin\n/#Competition','#MultiSeller\n/#Competition',\
                                 '#FewOffer\n/#Competition']]
dataframe_comp.columns=['Note','#Competition','#MultiWin','#ZeroWin','#MultiSeller',\
                        '#FewOffer']
print(tabulate(dataframe_comp,headers='keys', tablefmt='simple'))


Prbs per competition for 7 markets

Markets         Note            #Competition    #MultiWin    #ZeroWin    #MultiSeller    #FewOffer
--------------  ------------  --------------  -----------  ----------  --------------  -----------
France          Market1_data           79470         5798        2523            1706         6392
United Kingdom  Market2_data             718            0         359               0          610
Germany         Market3_data             723            0         188               1          699
Spain           Market4_data             992            0         410              37          894
India           Market5_data             445            0         222              10          442
Italy           Market6_data           16537          757         653             345         6647
US              Market7_data           13824            0         279               5        12736


In [168]:
## Save file
dataframe_prod.to_csv("./Prb/Pbl_market_product.csv",sep=',',encoding='utf-8')
dataframe_comp.to_csv("./Prb/Pbl_market_competition.csv",sep=',',encoding='utf-8')

In [169]:
np.save("./Prb/mk1_pd_sum",L1)
np.save("./Prb/mk2_pd_sum",L2)
np.save("./Prb/mk3_pd_sum",L3)
np.save("./Prb/mk4_pd_sum",L4)
np.save("./Prb/mk5_pd_sum",L5)
np.save("./Prb/mk6_pd_sum",L6)
np.save("./Prb/mk7_pd_sum",L7)
## can be loaded as: L1 = np.load("./Prb/mk1_pd_sum.npy")

## <font color='darkblue'>Binning products by number of competitions</font>
We bin products based on the number of competitions they have to study groups of products with little or intensive competition.

We use 4 bins for now:

[0-5] Products that have up to 5 competitions

[5-10] Products that have 6 to 10 competitions

[10-20] Products that have 11 to 20 competitions

[ > 20] Products that more than 20 competitions


In [170]:
df_1 = Market1_data.ix[:,['IsBuyBoxWinner','MarketplaceId','ProductId','CompetitionName','SellerId',\
                         'ListingPrice','ShippingPrice']]
df_2 = Market2_data.ix[:,['IsBuyBoxWinner','MarketplaceId','ProductId','CompetitionName','SellerId',\
                         'ListingPrice','ShippingPrice']]
df_3 = Market3_data.ix[:,['IsBuyBoxWinner','MarketplaceId','ProductId','CompetitionName','SellerId',\
                         'ListingPrice','ShippingPrice']]
df_4 = Market4_data.ix[:,['IsBuyBoxWinner','MarketplaceId','ProductId','CompetitionName','SellerId',\
                         'ListingPrice','ShippingPrice']]
df_5 = Market5_data.ix[:,['IsBuyBoxWinner','MarketplaceId','ProductId','CompetitionName','SellerId',\
                         'ListingPrice','ShippingPrice']]
df_6 = Market6_data.ix[:,['IsBuyBoxWinner','MarketplaceId','ProductId','CompetitionName','SellerId',\
                         'ListingPrice','ShippingPrice']]
df_7 = Market7_data.ix[:,['IsBuyBoxWinner','MarketplaceId','ProductId','CompetitionName','SellerId',\
                         'ListingPrice','ShippingPrice']]

In [171]:
def checkbin(df):
    groups=df.groupby('ProductId')
    bin1=0
    bin2=0
    bin3=0
    bin4=0
    for name,group in groups:
        k = len(group['CompetitionName'].unique())
        if(k<=5):
            bin1+=1
        if((k <= 10)&(k >5)):
            bin2+=1
        if((k <= 20)&(k >10)):
            bin3+=1
        if(k>20):
            bin4+=1
    return [bin1,bin2,bin3,bin4]

arr_checkbin_ = np.reshape([(checkbin(df_1)),(checkbin(df_2)),(checkbin(df_3)),(checkbin(df_4)),(checkbin(df_5)),\
 (checkbin(df_6)),(checkbin(df_7))],(7,4))

In [172]:
"""
========================================
Counting products per bin for 7 markets
========================================
"""
print(__doc__)
index = ['France','UK','Germany','Spain','India','Italy','US']
df_bin = pd.DataFrame([],index=index, columns=['#Product','[0-5]','[5-10]','[10-20]', '>20'])
df_bin.index.name = 'Markets'
df_bin["#Product"] = [p1,p2,p3,p4,p5,p6,p7]
df_bin["[0-5]"]= arr_checkbin_[:,0]
df_bin["[5-10]"]= arr_checkbin_[:,1]
df_bin["[10-20]"]= arr_checkbin_[:,2]
df_bin[">20"]= arr_checkbin_[:,3]
print(tabulate(df_bin,headers='keys', tablefmt='simple'))


Counting products per bin for 7 markets

Markets      #Product    [0-5]    [5-10]    [10-20]    >20
---------  ----------  -------  --------  ---------  -----
France            460      151        53         41    215
UK                117      105         2          2      8
Germany           130      111        11          2      6
Spain             163      137        14          4      8
India             134      126         6          1      1
Italy            1349      962       160         91    136
US                945      409       189        220    127


In the above table, for the France as an example, there are 460 products in total, while the amount of products in each bin are [151, 53, 41, 215] respectively. Also from that table, we can see the distribution of products per bins for all markets is highly placed in the bin [0-5]. It means that in those markets there are many products which have less than 5 competitions over the 9 months period covered by our dataset.

## <font color='darkblue'>Problems per bin</font>

In [173]:
## function to get data for bins
def getbins(df):
    groups = df.groupby(['ProductId'])
    bin1=pd.DataFrame([])
    bin2=pd.DataFrame([])
    bin3=pd.DataFrame([])
    bin4=pd.DataFrame([])
    l=len(groups)
    i=0
    for name,group in groups:
        k = len(group['CompetitionName'].unique())
        if(k<=5):
            bin1 = bin1.append(group,ignore_index=True)
        if((k <= 10)&(k >5)):
            bin2 = bin2.append(group,ignore_index=True)
        if((k <= 20)&(k >10)):
            bin3= bin3.append(group,ignore_index=True)
        if(k>20):
            bin4=bin4.append(group,ignore_index=True)
        i+=1
        print("Processing {:2.1%}".format(i/l), end="\r")
    print("\n Done!")
    return [bin1,bin2,bin3,bin4]

In [203]:
[mk1_bin1,mk1_bin2,mk1_bin3,mk1_bin4] = getbins(df_1)
[mk2_bin1,mk2_bin2,mk2_bin3,mk2_bin4] = getbins(df_2)
[mk3_bin1,mk3_bin2,mk3_bin3,mk3_bin4] = getbins(df_3)
[mk4_bin1,mk4_bin2,mk4_bin3,mk4_bin4] = getbins(df_4)
[mk5_bin1,mk5_bin2,mk5_bin3,mk5_bin4] = getbins(df_5)
[mk6_bin1,mk6_bin2,mk6_bin3,mk6_bin4] = getbins(df_6)
[mk7_bin1,mk7_bin2,mk7_bin3,mk7_bin4] = getbins(df_7)
clear_output()

## <font color='darkblue'>Market 1 : France</font>

In [175]:
## Market 1

## Bin1
B1L1 =[]
i=1
groups = mk1_bin1.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B1L1.append(t)
    i+=1
print("\n Done!")
B1L1 = np.reshape(B1L1,(l, 5))
B1SS1 = sum(B1L1)
B1S1 = sum(B1L1!=0)

## Bin2
B2L1 =[]
i=1
groups = mk1_bin2.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B2L1.append(t)
    i+=1
print("\n Done!")
B2L1 = np.reshape(B2L1,(l, 5))
B2SS1 = sum(B2L1)
B2S1 = sum(B2L1!=0)

## Bin3
B3L1 =[]
i=1
groups = mk1_bin3.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B3L1.append(t)
    i+=1
print("\n Done!")
B3L1 = np.reshape(B3L1,(l, 5))
B3SS1 = sum(B3L1)
B3S1 = sum(B3L1!=0)

## Bin4
B4L1 =[]
i=1
groups = mk1_bin4.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B4L1.append(t)
    i+=1
print("\n Done!")
B4L1 = np.reshape(B4L1,(l, 5))
B4SS1 = sum(B4L1)
B4S1 = sum(B4L1!=0)
clear_output()

In [176]:
C1A1 = np.reshape([B1SS1,B2SS1,B3SS1,B4SS1],(4,5))
C2A1 = np.reshape([B1S1,B2S1,B3S1,B4S1],(4,5))

In [177]:
"""
==============================================================
Counting products having problems per bin for market 1: France
==============================================================
"""
print(__doc__)
## insert to table
##  market 1: prbs per product
index = ['[0-5]','[5-10]','[10-20]', '>20']
df_mk1_prd_bin = pd.DataFrame([],index=index, columns=['#Product','#MultiWin','#ZeroWin',\
                                                   '#MultiSeller','#FewOffer',])
df_mk1_prd_bin.index.name = 'Bins'
df_mk1_prd_bin["#Product"] = ([len(mk1_bin1.groupby(['ProductId'])),len(mk1_bin2.groupby(['ProductId']))\
                         ,len(mk1_bin3.groupby(['ProductId'])),len(mk1_bin4.groupby(['ProductId']))])
df_mk1_prd_bin["#MultiWin"] = C2A1[:,1]
df_mk1_prd_bin["#ZeroWin"] = C2A1[:,2]
df_mk1_prd_bin["#MultiSeller"] = C2A1[:,3]
df_mk1_prd_bin["#FewOffer"] = C2A1[:,4]
print(tabulate(df_mk1_prd_bin,headers='keys', tablefmt='simple'))



Counting products having problems per bin for market 1: France

Bins       #Product    #MultiWin    #ZeroWin    #MultiSeller    #FewOffer
-------  ----------  -----------  ----------  --------------  -----------
[0-5]           151           11          52              21           98
[5-10]           53            2          17               8           34
[10-20]          41            3          13               9           21
>20             215           81          35              72           49


In [178]:
"""
================================================================
Counting competition having problems per bin for market 1: France
================================================================
"""
print(__doc__)
##  market 1: prbs per competition
df_mk1_comp_bin = pd.DataFrame([],index=index, columns=['#Competitions','#MultiWin','#ZeroWin',\
                                                   '#MultiSeller','#FewOffer',])
df_mk1_comp_bin.index.name = 'Bins'
df_mk1_comp_bin["#Competitions"] = C1A1[:,0]
df_mk1_comp_bin["#MultiWin"] = C1A1[:,1]
df_mk1_comp_bin["#ZeroWin"] = C1A1[:,2]
df_mk1_comp_bin["#MultiSeller"] = C1A1[:,3]
df_mk1_comp_bin["#FewOffer"] = C1A1[:,4]
print(tabulate(df_mk1_comp_bin,headers='keys', tablefmt='simple'))


Counting competition having problems per bin for market 1: France

Bins       #Competitions    #MultiWin    #ZeroWin    #MultiSeller    #FewOffer
-------  ---------------  -----------  ----------  --------------  -----------
[0-5]                278           18          74              30          181
[5-10]               412           10          38              46          236
[10-20]              593            5          53             109          265
>20                78187         5765        2358           11016         5710


## <font color='darkblue'>Market 2 : UK</font>

In [179]:
## Market 2

## Bin1
B1L2 =[]
i=1
groups = mk2_bin1.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B1L2.append(t)
    i+=1
print("\n Done!")
B1L2 = np.reshape(B1L2,(l, 5))
B1SS2 = sum(B1L2)
B1S2 = sum(B1L2!=0)

## Bin2
B2L2 =[]
i=1
groups = mk2_bin2.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B2L2.append(t)
    i+=1
print("\n Done!")
B2L2 = np.reshape(B2L2,(l, 5))
B2SS2 = sum(B2L2)
B2S2 = sum(B2L2!=0)

## Bin3
B3L2 =[]
i=1
groups = mk2_bin3.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B3L2.append(t)
    i+=1
print("\n Done!")
B3L2 = np.reshape(B3L2,(l, 5))
B3SS2 = sum(B3L2)
B3S2 = sum(B3L2!=0)

## Bin4
B4L2 =[]
i=1
groups = mk2_bin4.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B4L2.append(t)
    i+=1
print("\n Done!")
B4L2 = np.reshape(B4L2,(l, 5))
B4SS2 = sum(B4L2)
B4S2 = sum(B4L2!=0)
clear_output()

In [180]:
C1A2 = np.reshape([B1SS2,B2SS2,B3SS2,B4SS2],(4,5))
C2A2 = np.reshape([B1S2,B2S2,B3S2,B4S2],(4,5))

In [181]:
"""
============================================================
Counting products having problems per bin for market 2: UK
============================================================
"""
print(__doc__)
## insert to table
## market 2: prbs per product
index = ['[0-5]','[5-10]','[10-20]', '>20']
df_mk2_prd_bin = pd.DataFrame([],index=index, columns=['#Product','#MultiWin','#ZeroWin',\
                                                   '#MultiSeller','#FewOffer',])
df_mk2_prd_bin.index.name = 'Bins'
df_mk2_prd_bin["#Product"] = C2A2[:,0]
df_mk2_prd_bin["#MultiWin"] = C2A2[:,1]
df_mk2_prd_bin["#ZeroWin"] = C2A2[:,2]
df_mk2_prd_bin["#MultiSeller"] = C2A2[:,3]
df_mk2_prd_bin["#FewOffer"] = C2A2[:,4]
print(tabulate(df_mk2_prd_bin,headers='keys', tablefmt='simple'))



Counting products having problems per bin for market 2: UK

Bins       #Product    #MultiWin    #ZeroWin    #MultiSeller    #FewOffer
-------  ----------  -----------  ----------  --------------  -----------
[0-5]           105            0          81               4          105
[5-10]            2            0           2               0            2
[10-20]           2            0           1               0            2
>20               8            0           7               1            8


In [182]:
"""
==============================================================
Counting competitions having problems per bin for market 2: UK
==============================================================
"""
print(__doc__)
## ## market 2: prbs per competition
df_mk2_comp_bin = pd.DataFrame([],index=index, columns=['#Competitions','#MultiWin','#ZeroWin',\
                                                   '#MultiSeller','#FewOffer',])
df_mk2_comp_bin.index.name = 'Bins'
df_mk2_comp_bin["#Competitions"] = C1A2[:,0]
df_mk2_comp_bin["#MultiWin"] = C1A2[:,1]
df_mk2_comp_bin["#ZeroWin"] = C1A2[:,2]
df_mk2_comp_bin["#MultiSeller"] = C1A2[:,3]
df_mk2_comp_bin["#FewOffer"] = C1A2[:,4]
print(tabulate(df_mk2_comp_bin,headers='keys', tablefmt='simple'))



Counting competitions having problems per bin for market 2: UK

Bins       #Competitions    #MultiWin    #ZeroWin    #MultiSeller    #FewOffer
-------  ---------------  -----------  ----------  --------------  -----------
[0-5]                174            0         129               6          174
[5-10]                16            0          12               0           16
[10-20]               36            0          10               0           35
>20                  492            0         208              10          385


## <font color='darkblue'>Market 3 : Germany</font>

In [183]:
## Market 3

## Bin1
B1L3 =[]
i=1
groups = mk3_bin1.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B1L3.append(t)
    i+=1
print("\n Done!")
B1L3 = np.reshape(B1L3,(l, 5))
B1SS3 = sum(B1L3)
B1S3 = sum(B1L3!=0)

## Bin2
B2L3 =[]
i=1
groups = mk3_bin2.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B2L3.append(t)
    i+=1
print("\n Done!")
B2L3 = np.reshape(B2L3,(l, 5))
B2SS3 = sum(B2L3)
B2S3 = sum(B2L3!=0)

## Bin3
B3L3 =[]
i=1
groups = mk3_bin3.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B3L3.append(t)
    i+=1
print("\n Done!")
B3L3 = np.reshape(B3L3,(l, 5))
B3SS3 = sum(B3L3)
B3S3 = sum(B3L3!=0)

## Bin4
B4L3 =[]
i=1
groups = mk3_bin4.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B4L3.append(t)
    i+=1
print("\n Done!")
B4L3 = np.reshape(B4L3,(l, 5))
B4SS3 = sum(B4L3)
B4S3 = sum(B4L3!=0)
clear_output()

In [184]:
C1A3 = np.reshape([B1SS3,B2SS3,B3SS3,B4SS3],(4,5))
C2A3 = np.reshape([B1S3,B2S3,B3S3,B4S3],(4,5))

In [185]:
"""
===============================================================
Counting products having problems per bin for market 3: Germany
===============================================================
"""
print(__doc__)
## market 3: prbs per product
index = ['[0-5]','[5-10]','[10-20]', '>20']
df_mk3_prd_bin = pd.DataFrame([],index=index, columns=['#Product','#MultiWin','#ZeroWin',\
                                                   '#MultiSeller','#FewOffer',])
df_mk3_prd_bin.index.name = 'Bins'
df_mk3_prd_bin["#Product"] = C2A3[:,0]
df_mk3_prd_bin["#MultiWin"] = C2A3[:,1]
df_mk3_prd_bin["#ZeroWin"] = C2A3[:,2]
df_mk3_prd_bin["#MultiSeller"] = C2A3[:,3]
df_mk3_prd_bin["#FewOffer"] = C2A3[:,4]
print(tabulate(df_mk3_prd_bin,headers='keys', tablefmt='simple'))



Counting products having problems per bin for market 3: Germany

Bins       #Product    #MultiWin    #ZeroWin    #MultiSeller    #FewOffer
-------  ----------  -----------  ----------  --------------  -----------
[0-5]           111            0          97               3          111
[5-10]           11            0           9               1           11
[10-20]           2            0           2               1            2
>20               6            0           3               1            6


In [186]:
"""
===================================================================
Counting competitions having problems per bin for market 3: Germany
===================================================================
"""
print(__doc__)
## ## market 3: prbs per competition
df_mk3_comp_bin = pd.DataFrame([],index=index, columns=['#Competitions','#MultiWin','#ZeroWin',\
                                                   '#MultiSeller','#FewOffer',])
df_mk3_comp_bin.index.name = 'Bins'
df_mk3_comp_bin["#Competitions"] = C1A3[:,0]
df_mk3_comp_bin["#MultiWin"] = C1A3[:,1]
df_mk3_comp_bin["#ZeroWin"] = C1A3[:,2]
df_mk3_comp_bin["#MultiSeller"] = C1A3[:,3]
df_mk3_comp_bin["#FewOffer"] = C1A3[:,4]
print(tabulate(df_mk2_comp_bin,headers='keys', tablefmt='simple'))



Counting competitions having problems per bin for market 3: Germany

Bins       #Competitions    #MultiWin    #ZeroWin    #MultiSeller    #FewOffer
-------  ---------------  -----------  ----------  --------------  -----------
[0-5]                174            0         129               6          174
[5-10]                16            0          12               0           16
[10-20]               36            0          10               0           35
>20                  492            0         208              10          385


## <font color='darkblue'>Market 4 : Spain</font>

In [187]:
## Market 4

## Bin1
B1L4 =[]
i=1
groups = mk4_bin1.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B1L4.append(t)
    i+=1
print("\n Done!")
B1L4 = np.reshape(B1L4,(l, 5))
B1SS4 = sum(B1L4)
B1S4 = sum(B1L4!=0)

## Bin2
B2L4 =[]
i=1
groups = mk4_bin2.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B2L4.append(t)
    i+=1
print("\n Done!")
B2L4 = np.reshape(B2L4,(l, 5))
B2SS4 = sum(B2L4)
B2S4 = sum(B2L4!=0)

## Bin3
B3L4 =[]
i=1
groups = mk4_bin3.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B3L4.append(t)
    i+=1
print("\n Done!")
B3L4 = np.reshape(B3L4,(l, 5))
B3SS4 = sum(B3L4)
B3S4 = sum(B3L4!=0)

## Bin4
B4L4 =[]
i=1
groups = mk4_bin4.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B4L4.append(t)
    i+=1
print("\n Done!")
B4L4 = np.reshape(B4L4,(l, 5))
B4SS4 = sum(B4L4)
B4S4 = sum(B4L4!=0)
clear_output()

In [188]:
C1A4 = np.reshape([B1SS4,B2SS4,B3SS4,B4SS4],(4,5))
C2A4 = np.reshape([B1S4,B2S4,B3S4,B4S4],(4,5))

In [189]:
"""
===============================================================
Counting products having problems per bin for market 4: Spain
===============================================================
"""
print(__doc__)
## market 4: prbs per product
index = ['[0-5]','[5-10]','[10-20]', '>20']
df_mk4_prd_bin = pd.DataFrame([],index=index, columns=['#Product','#MultiWin','#ZeroWin',\
                                                   '#MultiSeller','#FewOffer',])
df_mk4_prd_bin.index.name = 'Bins'
df_mk4_prd_bin["#Product"] = C2A4[:,0]
df_mk4_prd_bin["#MultiWin"] = C2A4[:,1]
df_mk4_prd_bin["#ZeroWin"] = C2A4[:,2]
df_mk4_prd_bin["#MultiSeller"] = C2A4[:,3]
df_mk4_prd_bin["#FewOffer"] = C2A4[:,4]
print(tabulate(df_mk4_prd_bin,headers='keys', tablefmt='simple'))



Counting products having problems per bin for market 4: Spain

Bins       #Product    #MultiWin    #ZeroWin    #MultiSeller    #FewOffer
-------  ----------  -----------  ----------  --------------  -----------
[0-5]           137            0         107               9          137
[5-10]           14            0          12               2           14
[10-20]           4            0           4               1            4
>20               8            0           6               3            7


In [190]:
"""
===================================================================
Counting competitions having problems per bin for market 4: Spain
===================================================================
"""
print(__doc__)
## ## market 4: prbs per competition
df_mk4_comp_bin = pd.DataFrame([],index=index, columns=['#Competitions','#MultiWin','#ZeroWin',\
                                                   '#MultiSeller','#FewOffer',])
df_mk4_comp_bin.index.name = 'Bins'
df_mk4_comp_bin["#Competitions"] = C1A4[:,0]
df_mk4_comp_bin["#MultiWin"] = C1A4[:,1]
df_mk4_comp_bin["#ZeroWin"] = C1A4[:,2]
df_mk4_comp_bin["#MultiSeller"] = C1A4[:,3]
df_mk4_comp_bin["#FewOffer"] = C1A4[:,4]
print(tabulate(df_mk4_comp_bin,headers='keys', tablefmt='simple'))



Counting competitions having problems per bin for market 4: Spain

Bins       #Competitions    #MultiWin    #ZeroWin    #MultiSeller    #FewOffer
-------  ---------------  -----------  ----------  --------------  -----------
[0-5]                268            0         181              11          268
[5-10]               106            0          69              12          104
[10-20]               60            0          24               3           60
>20                  558            0         136             117          462


## <font color='darkblue'>Market 5 : India</font>

In [191]:
## Market 5

## Bin1
B1L5 =[]
i=1
groups = mk5_bin1.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B1L5.append(t)
    i+=1
print("\n Done!")
B1L5 = np.reshape(B1L5,(l, 5))
B1SS5 = sum(B1L5)
B1S5 = sum(B1L5!=0)

## Bin2
B2L5 =[]
i=1
groups = mk5_bin2.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B2L5.append(t)
    i+=1
print("\n Done!")
B2L5 = np.reshape(B2L5,(l, 5))
B2SS5 = sum(B2L5)
B2S5 = sum(B2L5!=0)

## Bin3
B3L5 =[]
i=1
groups = mk5_bin3.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B3L5.append(t)
    i+=1
print("\n Done!")
B3L5 = np.reshape(B3L5,(l, 5))
B3SS5 = sum(B3L5)
B3S5 = sum(B3L5!=0)

## Bin4
B4L5 =[]
i=1
groups = mk5_bin4.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B4L5.append(t)
    i+=1
print("\n Done!")
B4L5 = np.reshape(B4L5,(l, 5))
B4SS5 = sum(B4L5)
B4S5 = sum(B4L5!=0)
clear_output()

In [192]:
C1A5 = np.reshape([B1SS5,B2SS5,B3SS5,B4SS5],(4,5))
C2A5 = np.reshape([B1S5,B2S5,B3S5,B4S5],(4,5))

In [193]:
"""
===============================================================
Counting products having problems per bin for market 5: India
===============================================================
"""
print(__doc__)
## market 5: prbs per product
index = ['[0-5]','[5-10]','[10-20]', '>20']
df_mk5_prd_bin = pd.DataFrame([],index=index, columns=['#Product','#MultiWin','#ZeroWin',\
                                                   '#MultiSeller','#FewOffer',])
df_mk5_prd_bin.index.name = 'Bins'
df_mk5_prd_bin["#Product"] = C2A5[:,0]
df_mk5_prd_bin["#MultiWin"] = C2A5[:,1]
df_mk5_prd_bin["#ZeroWin"] = C2A5[:,2]
df_mk5_prd_bin["#MultiSeller"] = C2A5[:,3]
df_mk5_prd_bin["#FewOffer"] = C2A5[:,4]
print(tabulate(df_mk5_prd_bin,headers='keys', tablefmt='simple'))



Counting products having problems per bin for market 5: India

Bins       #Product    #MultiWin    #ZeroWin    #MultiSeller    #FewOffer
-------  ----------  -----------  ----------  --------------  -----------
[0-5]           126            0         104               4          126
[5-10]            6            0           5               2            6
[10-20]           1            0           1               0            1
>20               1            0           1               0            1


In [194]:
"""
===================================================================
Counting competitions having problems per bin for market 5: India
===================================================================
"""
print(__doc__)
## ## market 5: prbs per competition
df_mk5_comp_bin = pd.DataFrame([],index=index, columns=['#Competitions','#MultiWin','#ZeroWin',\
                                                   '#MultiSeller','#FewOffer',])
df_mk5_comp_bin.index.name = 'Bins'
df_mk5_comp_bin["#Competitions"] = C1A5[:,0]
df_mk5_comp_bin["#MultiWin"] = C1A5[:,1]
df_mk5_comp_bin["#ZeroWin"] = C1A5[:,2]
df_mk5_comp_bin["#MultiSeller"] = C1A5[:,3]
df_mk5_comp_bin["#FewOffer"] = C1A5[:,4]
print(tabulate(df_mk5_comp_bin,headers='keys', tablefmt='simple'))



Counting competitions having problems per bin for market 5: India

Bins       #Competitions    #MultiWin    #ZeroWin    #MultiSeller    #FewOffer
-------  ---------------  -----------  ----------  --------------  -----------
[0-5]                192            0         149               8          189
[5-10]                39            0          18               5           39
[10-20]               15            0           1               0           15
>20                  199            0          54               0          199


## <font color='darkblue'>Market 6 : Italy</font>

In [195]:
## Market 6

## Bin1
B1L6 =[]
i=1
groups = mk6_bin1.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B1L6.append(t)
    i+=1
print("\n Done!")
B1L6 = np.reshape(B1L6,(l, 5))
B1SS6 = sum(B1L6)
B1S6 = sum(B1L6!=0)

## Bin2
B2L6 =[]
i=1
groups = mk6_bin2.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B2L6.append(t)
    i+=1
print("\n Done!")
B2L6 = np.reshape(B2L6,(l, 5))
B2SS6 = sum(B2L6)
B2S6 = sum(B2L6!=0)

## Bin3
B3L6 =[]
i=1
groups = mk6_bin3.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B3L6.append(t)
    i+=1
print("\n Done!")
B3L6 = np.reshape(B3L6,(l, 5))
B3SS6 = sum(B3L6)
B3S6 = sum(B3L6!=0)

## Bin4
B4L6 =[]
i=1
groups = mk6_bin4.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B4L6.append(t)
    i+=1
print("\n Done!")
B4L6 = np.reshape(B4L6,(l, 5))
B4SS6 = sum(B4L6)
B4S6 = sum(B4L6!=0)
clear_output()

In [196]:
C1A6 = np.reshape([B1SS6,B2SS6,B3SS6,B4SS6],(4,5))
C2A6 = np.reshape([B1S6,B2S6,B3S6,B4S6],(4,5))

In [197]:
"""
===============================================================
Counting products having problems per bin for market 6: Italy
===============================================================
"""
print(__doc__)
## market 6: prbs per product
index = ['[0-5]','[5-10]','[10-20]', '>20']
df_mk6_prd_bin = pd.DataFrame([],index=index, columns=['#Product','#MultiWin','#ZeroWin',\
                                                   '#MultiSeller','#FewOffer',])
df_mk6_prd_bin.index.name = 'Bins'
df_mk6_prd_bin["#Product"] = C2A6[:,0]
df_mk6_prd_bin["#MultiWin"] = C2A6[:,1]
df_mk6_prd_bin["#ZeroWin"] = C2A6[:,2]
df_mk6_prd_bin["#MultiSeller"] = C2A6[:,3]
df_mk6_prd_bin["#FewOffer"] = C2A6[:,4]
print(tabulate(df_mk6_prd_bin,headers='keys', tablefmt='simple'))



Counting products having problems per bin for market 6: Italy

Bins       #Product    #MultiWin    #ZeroWin    #MultiSeller    #FewOffer
-------  ----------  -----------  ----------  --------------  -----------
[0-5]           962           17         181              66          603
[5-10]          160           13           8              14           73
[10-20]          91            8           5              10           44
>20             136           19          15               6           65


In [198]:
"""
===================================================================
Counting competitions having problems per bin for market 6: Italy
===================================================================
"""
print(__doc__)
## ## market 6: prbs per competition
df_mk6_comp_bin = pd.DataFrame([],index=index, columns=['#Competitions','#MultiWin','#ZeroWin',\
                                                   '#MultiSeller','#FewOffer',])
df_mk6_comp_bin.index.name = 'Bins'
df_mk6_comp_bin["#Competitions"] = C1A6[:,0]
df_mk6_comp_bin["#MultiWin"] = C1A6[:,1]
df_mk6_comp_bin["#ZeroWin"] = C1A6[:,2]
df_mk6_comp_bin["#MultiSeller"] = C1A6[:,3]
df_mk6_comp_bin["#FewOffer"] = C1A6[:,4]
print(tabulate(df_mk6_comp_bin,headers='keys', tablefmt='simple'))



Counting competitions having problems per bin for market 6: Italy

Bins       #Competitions    #MultiWin    #ZeroWin    #MultiSeller    #FewOffer
-------  ---------------  -----------  ----------  --------------  -----------
[0-5]               1933           28         232             146         1135
[5-10]              1224           25          27              75          510
[10-20]             1326           18          29             126          524
>20                12054          686         365             182         4478


## <font color='darkblue'>Market 7 : US</font>

In [199]:
## Market 7

## Bin1
B1L7 =[]
i=1
groups = mk7_bin1.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B1L7.append(t)
    i+=1
print("\n Done!")
B1L7 = np.reshape(B1L7,(l, 5))
B1SS7 = sum(B1L7)
B1S7 = sum(B1L7!=0)

## Bin2
B2L7 =[]
i=1
groups = mk7_bin2.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B2L7.append(t)
    i+=1
print("\n Done!")
B2L7 = np.reshape(B2L7,(l, 5))
B2SS7 = sum(B2L7)
B2S7 = sum(B2L7!=0)

## Bin3
B3L7 =[]
i=1
groups = mk7_bin3.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B3L7.append(t)
    i+=1
print("\n Done!")
B3L7 = np.reshape(B3L7,(l, 5))
B3SS7 = sum(B3L7)
B3S7 = sum(B3L7!=0)

## Bin4
B4L7 =[]
i=1
groups = mk7_bin4.groupby(['ProductId'])
l = len(groups)
for competition_name, group in (groups):
    t=checkPrbl(group)
    sys.stdout.write("\r Processing in product {0}/{1}".format(i,l))
    sys.stdout.flush()
    B4L7.append(t)
    i+=1
print("\n Done!")
B4L7 = np.reshape(B4L7,(l, 5))
B4SS7 = sum(B4L7)
B4S7 = sum(B4L7!=0)
clear_output()

In [200]:
C1A7 = np.reshape([B1SS7,B2SS7,B3SS7,B4SS7],(4,5))
C2A7 = np.reshape([B1S7,B2S7,B3S7,B4S7],(4,5))

In [201]:
"""
===============================================================
Counting products having problems per bin for market 7: US
===============================================================
"""
print(__doc__)
## market 7: prbs per product
index = ['[0-5]','[5-10]','[10-20]', '>20']
df_mk7_prd_bin = pd.DataFrame([],index=index, columns=['#Product','#MultiWin','#ZeroWin',\
                                                   '#MultiSeller','#FewOffer',])
df_mk7_prd_bin.index.name = 'Bins'
df_mk7_prd_bin["#Product"] = C2A7[:,0]
df_mk7_prd_bin["#MultiWin"] = C2A7[:,1]
df_mk7_prd_bin["#ZeroWin"] = C2A7[:,2]
df_mk7_prd_bin["#MultiSeller"] = C2A7[:,3]
df_mk7_prd_bin["#FewOffer"] = C2A7[:,4]
print(tabulate(df_mk7_prd_bin,headers='keys', tablefmt='simple'))



Counting products having problems per bin for market 7: US

Bins       #Product    #MultiWin    #ZeroWin    #MultiSeller    #FewOffer
-------  ----------  -----------  ----------  --------------  -----------
[0-5]           409            0          94               2          407
[5-10]          189            0           6               0          188
[10-20]         220            0           6               4          217
>20             127            0           9               3          124


In [202]:
"""
===================================================================
Counting competitions having problems per bin for market 7: US
===================================================================
"""
print(__doc__)
## ## market 7: prbs per competition
df_mk7_comp_bin = pd.DataFrame([],index=index, columns=['#Competitions','#MultiWin','#ZeroWin',\
                                                   '#MultiSeller','#FewOffer',])
df_mk7_comp_bin.index.name = 'Bins'
df_mk7_comp_bin["#Competitions"] = C1A7[:,0]
df_mk7_comp_bin["#MultiWin"] = C1A7[:,1]
df_mk7_comp_bin["#ZeroWin"] = C1A7[:,2]
df_mk7_comp_bin["#MultiSeller"] = C1A7[:,3]
df_mk7_comp_bin["#FewOffer"] = C1A7[:,4]
print(tabulate(df_mk7_comp_bin,headers='keys', tablefmt='simple'))



Counting competitions having problems per bin for market 7: US

Bins       #Competitions    #MultiWin    #ZeroWin    #MultiSeller    #FewOffer
-------  ---------------  -----------  ----------  --------------  -----------
[0-5]               1046            0         165               6         1020
[5-10]              1428            0          29               0         1410
[10-20]             3166            0          17              32         3113
>20                 8184            0          68             584         7193


# <font color='darkblue'>Conclusions</font>

We have seen an analysis of the XML dataset provided, for each market covered.

Questions:

1. What are good strategies to deal with the problems identified? 

    * why do some problems happen? (XML format error)

    * dropping data not always a good idea as it leads to high data loss

2. Which market should we focus on first? 
    * the larger markets: France, Italy, US or all markets?

3. Which products should we focus on first?
    * focus on products with little competition ([0-5] bin), medium or high competition?
